# GIS Data Science for Climate in Nepal

In [ ]:
!pip install pandas geopandas matplotlib seaborn folium fiona


In [ ]:
# Import necessary libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium


## Data Collection and Preparation

### Reading Administrative Boundaries Shapefile into GeoDataFrame

In [ ]:
import geopandas as gpd

# Define the file path (only specify the .shp file)
shapefile_path = r"data\nepal_shapefiles\local_unit.shp"

# Load the shapefile into a GeoDataFrame
gdf = gpd.read_file(shapefile_path)

# Display basic info
print(gdf.info())

# Preview the first few rows
gdf.head()


In [ ]:
# Check for Missing Values
print(gdf.isnull().sum())


In [ ]:
# Count Administrative Units by Province
gdf['Province'].value_counts()


In [ ]:
# Check the current CRS
print(gdf.crs)

# If needed, convert to WGS84 (latitude/longitude)
if gdf.crs != "EPSG:4326":
    gdf = gdf.to_crs(epsg=4326)

In [ ]:
import matplotlib.pyplot as plt

#Visualize the shapefile
gdf.plot(figsize=(10, 8), edgecolor='black', color='lightblue')
plt.title("Administrative Boundaries of Nepal")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


### Vector Data Visualization Using Matplotlib and GeoPandas

### Interactive Map using Folium

In [ ]:
! pip install rasterio rioxarray geopandas matplotlib numpy


In [ ]:
from IPython.display import display

# Define a map centered at an average location
m = folium.Map(location=[20, 0], zoom_start=2)

# Add markers with correct popup syntax
folium.Marker(location=[27.7103, 85.3222], popup="Kathmandu, Nepal").add_to(m)
folium.Marker(location=[40.712776, -74.005974], popup="New York, USA").add_to(m)

# Display the map in Jupyter Notebook
display(m)


In [ ]:
import geopandas as gpd
import folium

# Load the shapefile
shapefile_path = "data/nepal_shapefiles/local_unit.shp"
gdf = gpd.read_file(shapefile_path)

# Ensure the GeoDataFrame has a CRS
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)  # Assuming the original CRS is WGS 84

# Convert to projected CRS (e.g., UTM Zone 44N for Nepal)
gdf_projected = gdf.to_crs(epsg=32644)  # Use EPSG:32644 for Nepal (UTM Zone 44N)

# Compute centroid in projected CRS, then transform back to EPSG:4326
centroid_geom = gdf_projected.geometry.centroid.to_crs(epsg=4326)

# Calculate mean latitude and longitude for better centering
center = [centroid_geom.y.mean(), centroid_geom.x.mean()]

# Create a Folium map centered on Nepal
m = folium.Map(location=center, zoom_start=6)

# Add the shapefile as a GeoJSON layer
folium.GeoJson(gdf, name="Nepal Boundaries").add_to(m)

# Display the map
m


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Load the shapefile
shapefile_path = "data/nepal_shapefiles/local_unit.shp"
gdf = gpd.read_file(shapefile_path)

# Plot the shapefile
fig, ax = plt.subplots(figsize=(10, 5))
gdf.plot(ax=ax, cmap="viridis", edgecolor="black")

# Add title and show the plot
plt.title("Local Unit Boundaries of Nepal")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


## Visualization of Raster Data

In [ ]:
# Install rasterio and numpy
!pip install rasterio numpy rioxarray

In [ ]:
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.mask import mask


In [ ]:
import rasterio

# Define file paths for raster data
raster_paths = {
    "Precipitation 2020": "data/nepal_climate_rasters/nepal_precipitation_2020.tif",
    "Precipitation 2050": "data/nepal_climate_rasters/nepal_precipitation_2050.tif",
    "Temperature 2020": "data/nepal_climate_rasters/nepal_temperature_2020.tif",
    "Temperature 2050": "data/nepal_climate_rasters/nepal_temperature_2050.tif",
}

# Dictionary to store raster data and metadata
raster_data = {}

# Loop through files and read data
for key, path in raster_paths.items():
    try:
        with rasterio.open(path) as src:
            raster_data[key] = {
                "array": src.read(1),  # Read the first band
                "meta": src.meta  # Store metadata
            }
        print(f"Successfully loaded: {key}")
    except Exception as e:
        print(f"Error loading {key}: {e}")

# Example: Accessing the Temperature 2020 raster
temp_2020 = raster_data["Temperature 2020"]["array"]
temp_2020_meta = raster_data["Temperature 2020"]["meta"]

# Example: Checking metadata
print(temp_2020_meta)


In [ ]:
import matplotlib.pyplot as plt
from rasterio.plot import show

# Plot Temperature 2020
fig, ax = plt.subplots(figsize=(12, 8))

# Show raster using rasterio's show function
img = show(temp_2020, transform=temp_2020_meta["transform"], cmap="RdYlBu_r", ax=ax)

# Add title
plt.title("Temperature Nepal - 2020")

# Corrected colorbar implementation
cbar = plt.colorbar(img.get_images()[0], ax=ax, label="Temperature (°C)")

# Show the plot
plt.show()


In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterio.plot import show

# Define file path
temp_2050_path = "data/nepal_climate_rasters/nepal_temperature_2050.tif"

# Load the raster data
with rasterio.open(temp_2050_path) as src:
    temp_2050 = src.read(1)  # Read first band
    temp_2050_meta = src.meta  # Get metadata
    temp_2050[temp_2050 == src.nodata] = np.nan  # Replace NoData with NaN

# Plot Temperature 2050
fig, ax = plt.subplots(figsize=(12, 8))

# Show raster using rasterio's function
img = show(temp_2050, transform=temp_2050_meta["transform"], cmap="RdYlBu_r", ax=ax)

# Add title
plt.title("Projected Temperature Nepal - 2050")

# Corrected colorbar implementation
cbar = plt.colorbar(img.get_images()[0], ax=ax, label="Temperature (°C)")

# Show the plot
plt.show()


In [ ]:
import rasterio
import matplotlib.pyplot as plt

# Define file path
preci_2020_path = "data/nepal_climate_rasters/nepal_precipitation_2020.tif"

# Load raster data
with rasterio.open(preci_2020_path) as src:
    preci_2020 = src.read(1)  # Read first band
    preci_2020_meta = src.meta  # Store metadata

# Plot Precipitation 2020
fig, ax = plt.subplots(figsize=(12, 8))

# Display raster
img = ax.imshow(preci_2020, cmap="PuBuGn")

# Add title and labels
plt.title("Precipitation Nepal - 2020")
cbar = plt.colorbar(img, ax=ax, label="Precipitation (mm)")

# Show the plot
plt.show()


In [ ]:
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show

# Define file path
preci_2050_path = "data/nepal_climate_rasters/nepal_precipitation_2050.tif"

# Load raster data
with rasterio.open(preci_2050_path) as src:
    preci_2050 = src.read(1)  # Read first band
    preci_2050_meta = src.meta  # Store metadata

# Plot Precipitation 2050
fig, ax = plt.subplots(figsize=(12, 8))

# Show raster using rasterio's function
img = show(preci_2050, transform=preci_2050_meta["transform"], cmap="PuBuGn", ax=ax)

# Add title
plt.title("Projected Precipitation Nepal - 2050")

# Corrected colorbar implementation
cbar = plt.colorbar(ax.images[0], ax=ax, label="Precipitation (mm)")

# Show the plot
plt.show()


In [ ]:
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.mask import mask
from pathlib import Path

# Define file paths
shapefile_path = Path("data/nepal_shapefiles/local_unit.shp")
temp_path_2020 = Path("data/nepal_climate_rasters/nepal_temperature_2020.tif")
clipped_raster_path = Path("data/nepal_climate_rasters/nepal_temperature_2020_clipped.tif")

# Check if files exist
if not shapefile_path.exists():
    raise FileNotFoundError(f"Shapefile not found: {shapefile_path}")
if not temp_path_2020.exists():
    raise FileNotFoundError(f"Raster file not found: {temp_path_2020}")

# Load shapefile
gdf = gpd.read_file(shapefile_path)

# Open raster file (Temperature 2020)
with rasterio.open(temp_path_2020) as src:
    raster_crs = src.crs  # Get raster CRS
    if raster_crs is None:
        raise ValueError("Raster CRS is not defined.")

    print(f"Raster CRS: {raster_crs}")
    print(f"Shapefile CRS (before transformation): {gdf.crs}")

    # Ensure shapefile CRS matches the raster CRS
    if gdf.crs is None:
        gdf.set_crs(raster_crs, inplace=True)
    elif gdf.crs != raster_crs:
        gdf = gdf.to_crs(raster_crs)

    print(f"Shapefile CRS (after transformation): {gdf.crs}")

    # Extract geometries for masking
    shapes = [geom for geom in gdf.geometry]

    # Clip the raster
    clipped_raster, clipped_transform = mask(src, shapes, crop=True)

    # Copy metadata and update it
    clipped_meta = src.meta.copy()
    clipped_meta.update({
        "transform": clipped_transform,
        "width": clipped_raster.shape[2],
        "height": clipped_raster.shape[1],
        "count": 1,
        "nodata": src.nodata if src.nodata is not None else -9999,  # Handle NoData
    })

# Save the clipped raster
with rasterio.open(clipped_raster_path, "w", **clipped_meta) as dst:
    dst.write(clipped_raster[0], 1)  # Write only the first band

print(f"Clipped raster saved at: {clipped_raster_path}")

# Plot the clipped raster with better colors
plt.figure(figsize=(10, 6))
img = plt.imshow(clipped_raster[0], cmap="RdYlBu_r", interpolation="nearest")
plt.colorbar(img, label="Temperature (°C)")
plt.title("Clipped Nepal Temperature 2020")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


In [ ]:
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.mask import mask
from pathlib import Path

# Define paths
shapefile_path = Path("data/nepal_shapefiles/local_unit.shp")
temp_path_2050 = Path("data/nepal_climate_rasters/nepal_temperature_2050.tif")
clipped_raster_path = Path("data/nepal_climate_rasters/nepal_temperature_2050_clipped.tif")

# Check if shapefile exists
if not shapefile_path.exists():
    raise FileNotFoundError(f"Shapefile not found: {shapefile_path}")

# Load shapefile
gdf = gpd.read_file(shapefile_path)

# Open raster file (Temperature 2050)
with rasterio.open(temp_path_2050) as src:
    raster_crs = src.crs  # Get raster CRS
    if raster_crs is None:
        raise ValueError("Raster CRS is not defined.")

    print(f"Raster CRS: {raster_crs}")
    print(f"Shapefile CRS (before transformation): {gdf.crs}")

    # Ensure shapefile has a CRS before transformation
    if gdf.crs is None:
        gdf.set_crs(raster_crs, inplace=True)
    elif gdf.crs != raster_crs:
        gdf = gdf.to_crs(raster_crs)

    print(f"Shapefile CRS (after transformation): {gdf.crs}")

    # Extract geometries for masking
    shapes = [geom for geom in gdf.geometry]

    # Clip the raster
    clipped_raster, clipped_transform = mask(src, shapes, crop=True)

    # Check shape of clipped raster
    print(f"Clipped Raster Shape: {clipped_raster.shape}")  # (bands, height, width)

    # Update metadata
    clipped_meta = src.meta.copy()
    clipped_meta.update({
        "transform": clipped_transform,
        "width": clipped_raster.shape[2],
        "height": clipped_raster.shape[1],
        "count": 1,  # Ensure correct band count
        "nodata": src.nodata if src.nodata is not None else -9999  # Handle NoData values
    })

# Save the clipped raster
with rasterio.open(clipped_raster_path, "w", **clipped_meta) as dst:
    dst.write(clipped_raster[0], 1)  # Write only the first band

print(f"Clipped raster saved at: {clipped_raster_path}")

# Plot the clipped raster
plt.figure(figsize=(10, 6))
img = plt.imshow(clipped_raster[0], cmap="RdYlBu_r", interpolation="nearest")
plt.colorbar(img, label="Temperature (°C)")
plt.title("Clipped Nepal Temperature 2050")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


In [ ]:
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from rasterio.mask import mask
from pathlib import Path

# Define paths using Path (for cross-platform compatibility)
shapefile_path = Path("data/nepal_shapefiles/local_unit.shp")
preci_path_2020 = Path("data/nepal_climate_rasters/nepal_precipitation_2020.tif")
clipped_raster_path = Path("data/nepal_climate_rasters/nepal_precipitation_2020_clipped.tif")

# Check if files exist before proceeding
if not shapefile_path.exists():
    raise FileNotFoundError(f"Shapefile not found: {shapefile_path}")
if not preci_path_2020.exists():
    raise FileNotFoundError(f"Raster file not found: {preci_path_2020}")

# Load shapefile
gdf = gpd.read_file(shapefile_path)

# Open raster file (Precipitation 2020)
with rasterio.open(preci_path_2020) as src:
    raster_crs = src.crs  # Get raster CRS
    if raster_crs is None:
        raise ValueError("Raster CRS is not defined.")

    print(f"Raster CRS: {raster_crs}")
    print(f"Shapefile CRS (before transformation): {gdf.crs}")

    # Ensure shapefile CRS is defined and matches raster
    if gdf.crs is None:
        print("Shapefile CRS is missing. Setting it to match raster CRS.")
        gdf.set_crs(raster_crs, inplace=True)

    if gdf.crs != raster_crs:
        gdf = gdf.to_crs(raster_crs)

    print(f"Shapefile CRS (after transformation): {gdf.crs}")

    # Extract geometries
    shapes = [geom for geom in gdf.geometry]

    # Clip the raster
    clipped_raster, clipped_transform = mask(src, shapes, crop=True)

    # Check shape of clipped raster
    print(f"Clipped Raster Shape: {clipped_raster.shape}")  # (bands, height, width)

    # Handle NoData values
    nodata_value = src.nodata if src.nodata is not None else -9999

    # Update metadata
    clipped_meta = src.meta.copy()
    clipped_meta.update({
        "transform": clipped_transform,
        "width": clipped_raster.shape[2],
        "height": clipped_raster.shape[1],
        "count": 1,
        "nodata": nodata_value  # Explicitly set NoData value
    })

# Save the clipped raster
with rasterio.open(clipped_raster_path, "w", **clipped_meta) as dst:
    dst.write(clipped_raster[0], 1)  # Write only the first band

print(f"Clipped raster saved at: {clipped_raster_path}")

# Prepare raster for visualization (handle NoData values separately)
clipped_raster_viz = clipped_raster.copy()
if nodata_value is not None:
    clipped_raster_viz[clipped_raster_viz == nodata_value] = np.nan  # Set NoData values to NaN for display

# Plot the clipped raster
plt.figure(figsize=(10, 6))
img = plt.imshow(clipped_raster_viz[0], cmap="Blues", interpolation="nearest",
                 vmin=np.nanmin(clipped_raster_viz[0]), vmax=np.nanmax(clipped_raster_viz[0]))  # Avoid NoData distortion
plt.colorbar(img, label="Precipitation (mm)")
plt.title("Clipped Nepal Precipitation 2020")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


In [ ]:
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from rasterio.mask import mask
from pathlib import Path

# Define paths using Path (for cross-platform compatibility)
shapefile_path = Path("data/nepal_shapefiles/local_unit.shp")
preci_path_2050 = Path("data/nepal_climate_rasters/nepal_precipitation_2050.tif")
clipped_raster_path = Path("data/nepal_climate_rasters/nepal_precipitation_2050_clipped.tif")

# Check if files exist before proceeding
if not shapefile_path.exists():
    raise FileNotFoundError(f"Shapefile not found: {shapefile_path}")
if not preci_path_2050.exists():
    raise FileNotFoundError(f"Raster file not found: {preci_path_2050}")

# Load shapefile
gdf = gpd.read_file(shapefile_path)

# Open raster file (Precipitation 2050)
with rasterio.open(preci_path_2050) as src:
    raster_crs = src.crs  # Get raster CRS
    if raster_crs is None:
        raise ValueError("Raster CRS is not defined.")

    print(f"Raster CRS: {raster_crs}")
    print(f"Shapefile CRS (before transformation): {gdf.crs}")

    # Ensure shapefile CRS is defined and matches raster
    if gdf.crs is None:
        print("Shapefile CRS is missing. Setting it to match raster CRS.")
        gdf.set_crs(raster_crs, inplace=True)

    if gdf.crs != raster_crs:
        gdf = gdf.to_crs(raster_crs)

    print(f"Shapefile CRS (after transformation): {gdf.crs}")

    # Extract geometries
    shapes = [geom for geom in gdf.geometry]

    # Clip the raster
    clipped_raster, clipped_transform = mask(src, shapes, crop=True)

    # Check shape of clipped raster
    print(f"Clipped Raster Shape: {clipped_raster.shape}")  # (bands, height, width)

    # Handle NoData values
    nodata_value = src.nodata if src.nodata is not None else -9999

    # Update metadata
    clipped_meta = src.meta.copy()
    clipped_meta.update({
        "transform": clipped_transform,
        "width": clipped_raster.shape[2],
        "height": clipped_raster.shape[1],
        "count": 1,
        "nodata": nodata_value  # Explicitly set NoData value
    })

# Save the clipped raster
with rasterio.open(clipped_raster_path, "w", **clipped_meta) as dst:
    dst.write(clipped_raster[0], 1)  # Write only the first band

print(f"Clipped raster saved at: {clipped_raster_path}")

# Prepare raster for visualization (handle NoData values separately)
clipped_raster_viz = clipped_raster.copy()
if nodata_value is not None:
    clipped_raster_viz[clipped_raster_viz == nodata_value] = np.nan  # Set NoData values to NaN for display

# Plot the clipped raster
plt.figure(figsize=(10, 6))
img = plt.imshow(clipped_raster_viz[0], cmap="Blues", interpolation="nearest",
                 vmin=np.nanmin(clipped_raster_viz[0]), vmax=np.nanmax(clipped_raster_viz[0]))  # Avoid NoData distortion
plt.colorbar(img, label="Precipitation (mm)")
plt.title("Clipped Nepal Precipitation 2050")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()


## Exploratory Data Analysis (EDA)

### Compute Basic Statistics

In [ ]:
!pip install scikit-learn
!pip install scipy

In [ ]:
import numpy as np
import rasterio
from pathlib import Path

def compute_statistics(data):
    """Computes basic statistics for a given raster dataset."""
    if data.size == 0 or np.isnan(data).all():
        return {"Mean": "N/A", "Median": "N/A", "Min": "N/A", "Max": "N/A", "SD": "N/A"}
    
    return {
        "Mean": np.nanmean(data),
        "Median": np.nanmedian(data),
        "Min": np.nanmin(data),
        "Max": np.nanmax(data),
        "SD": np.nanstd(data)
    }

def load_raster(path):
    """Loads raster data and replaces NoData values with NaN."""
    path = Path(path)
    if not path.exists():
        print(f"Warning: Raster file not found: {path}")
        return np.array([])  # Return empty array to handle missing files safely
    
    with rasterio.open(path) as src:
        data = src.read(1)
        nodata_value = src.nodata
        if nodata_value is not None:
            data = np.where(data == nodata_value, np.nan, data)
    return data

# Define file paths
raster_paths = {
    "Precipitation 2020": Path("data/nepal_climate_rasters/nepal_precipitation_2020.tif"),
    "Precipitation 2050": Path("data/nepal_climate_rasters/nepal_precipitation_2050.tif"),
    "Temperature 2020": Path("data/nepal_climate_rasters/nepal_temperature_2020.tif"),
    "Temperature 2050": Path("data/nepal_climate_rasters/nepal_temperature_2050.tif"),
}

# Load raster datasets and compute statistics
data_arrays = {}
results = {}

for label, path in raster_paths.items():
    data = load_raster(path)
    data_arrays[label] = data
    results[label] = compute_statistics(data)

# Print statistics
print("\nClimate Raster Statistics:")
for label, stats in results.items():
    print(f"\n{label}:")
    for key, value in stats.items():
        print(f"  - {key}: {value if isinstance(value, str) else round(value, 2)}")

# Compute correlations
def compute_correlation(data1, data2):
    """Computes Pearson correlation between two datasets, ignoring NaNs."""
    if data1.size == 0 or data2.size == 0:
        return "N/A"
    
    mask = ~np.isnan(data1) & ~np.isnan(data2)
    if np.sum(mask) == 0:
        return "N/A"
    
    return np.corrcoef(data1[mask], data2[mask])[0, 1]

correlations = {
    "Temp 2020 & Precip 2020": compute_correlation(data_arrays["Temperature 2020"], data_arrays["Precipitation 2020"]),
    "Temp 2050 & Precip 2050": compute_correlation(data_arrays["Temperature 2050"], data_arrays["Precipitation 2050"]),
    "Temp 2020 & Temp 2050": compute_correlation(data_arrays["Temperature 2020"], data_arrays["Temperature 2050"]),
    "Precip 2020 & Precip 2050": compute_correlation(data_arrays["Precipitation 2020"], data_arrays["Precipitation 2050"]),
}

# Print correlation results
print("\nCorrelation Analysis (Pearson's r):")
for label, corr in correlations.items():
    print(f"  - {label}: {corr if isinstance(corr, str) else round(corr, 2)}")


### Histograms

#### Histograms for Precipitation and Temperature (2020 and 2050)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Load raster data
precip_2020 = load_raster("data/nepal_climate_rasters/nepal_precipitation_2020.tif")
precip_2050 = load_raster("data/nepal_climate_rasters/nepal_precipitation_2050.tif")
temp_2020 = load_raster("data/nepal_climate_rasters/nepal_temperature_2020.tif")
temp_2050 = load_raster("data/nepal_climate_rasters/nepal_temperature_2050.tif")

# Flatten and remove NaN values
precip_2020_values = precip_2020[~np.isnan(precip_2020)].flatten()
precip_2050_values = precip_2050[~np.isnan(precip_2050)].flatten()
temp_2020_values = temp_2020[~np.isnan(temp_2020)].flatten()
temp_2050_values = temp_2050[~np.isnan(temp_2050)].flatten()

# Create figure
plt.figure(figsize=(12, 10))

# Precipitation Histogram 2020
plt.subplot(2, 2, 1)
sns.histplot(precip_2020_values, bins="auto", kde=True, color="royalblue")
plt.xlabel("Precipitation (mm)")
plt.ylabel("Frequency")
plt.title("Precipitation Histogram 2020")

# Precipitation Histogram 2050
plt.subplot(2, 2, 2)
sns.histplot(precip_2050_values, bins="auto", kde=True, color="darkorange")
plt.xlabel("Precipitation (mm)")
plt.ylabel("Frequency")
plt.title("Precipitation Histogram 2050")

# Temperature Histogram 2020
plt.subplot(2, 2, 3)
sns.histplot(temp_2020_values, bins="auto", kde=True, color="darkred")
plt.xlabel("Temperature (°C)")
plt.ylabel("Frequency")
plt.title("Temperature Histogram 2020")

# Temperature Histogram 2050
plt.subplot(2, 2, 4)
sns.histplot(temp_2050_values, bins="auto", kde=True, color="goldenrod")
plt.xlabel("Temperature (°C)")
plt.ylabel("Frequency")
plt.title("Temperature Histogram 2050")

plt.tight_layout()
plt.show()


#### Histograms (2020 vs. 2050)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Load raster data
precip_2020 = load_raster("data/nepal_climate_rasters/nepal_precipitation_2020.tif")
precip_2050 = load_raster("data/nepal_climate_rasters/nepal_precipitation_2050.tif")
temp_2020 = load_raster("data/nepal_climate_rasters/nepal_temperature_2020.tif")
temp_2050 = load_raster("data/nepal_climate_rasters/nepal_temperature_2050.tif")

# Flatten and remove NaN values
precip_2020_values = precip_2020[~np.isnan(precip_2020)].flatten()
precip_2050_values = precip_2050[~np.isnan(precip_2050)].flatten()
temp_2020_values = temp_2020[~np.isnan(temp_2020)].flatten()
temp_2050_values = temp_2050[~np.isnan(temp_2050)].flatten()

# Create figure
plt.figure(figsize=(12, 5))

# Precipitation Histogram (2020 vs. 2050)
plt.subplot(1, 2, 1)
sns.histplot(precip_2020_values, bins="auto", kde=True, color="royalblue", label="2020", alpha=0.6)
sns.histplot(precip_2050_values, bins="auto", kde=True, color="darkorange", label="2050", alpha=0.6)
plt.xlabel("Precipitation (mm)")
plt.ylabel("Frequency")
plt.title("Precipitation Histogram (2020 vs. 2050)")
plt.legend()

# Temperature Histogram (2020 vs. 2050)
plt.subplot(1, 2, 2)
sns.histplot(temp_2020_values, bins="auto", kde=True, color="darkred", label="2020", alpha=0.6)
sns.histplot(temp_2050_values, bins="auto", kde=True, color="goldenrod", label="2050", alpha=0.6)
plt.xlabel("Temperature (°C)")
plt.ylabel("Frequency")
plt.title("Temperature Histogram (2020 vs. 2050)")
plt.legend()

plt.tight_layout()
plt.show()


### Boxplots

#### Boxplots for Precipitation and Temperature (2020 vs. 2050)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Create DataFrames for proper Seaborn visualization
df_precip = pd.DataFrame({
    "Precipitation 2020": precip_2020_values,
    "Precipitation 2050": precip_2050_values
})

df_temp = pd.DataFrame({
    "Temperature 2020": temp_2020_values,
    "Temperature 2050": temp_2050_values
})

# Plot boxplots
plt.figure(figsize=(12, 6))

# Precipitation Box Plot (2020 vs. 2050)
plt.subplot(1, 2, 1)
sns.boxplot(data=df_precip, palette=["royalblue", "darkorange"])
plt.ylabel("Precipitation (mm)")
plt.title("Precipitation Box Plot (2020 vs. 2050)")

# Temperature Box Plot (2020 vs. 2050)
plt.subplot(1, 2, 2)
sns.boxplot(data=df_temp, palette=["darkred", "goldenrod"])
plt.ylabel("Temperature (°C)")
plt.title("Temperature Box Plot (2020 vs. 2050)")

plt.tight_layout()
plt.show()


### Violin Plot

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Flattening the data and creating labels
data_2020 = pd.DataFrame({
    'Value': precip_2020_values.tolist() + temp_2020_values.tolist(),
    'Variable': ['Precipitation'] * len(precip_2020_values) + ['Temperature'] * len(temp_2020_values),
    'Year': ['2020'] * (len(precip_2020_values) + len(temp_2020_values))
})

data_2050 = pd.DataFrame({
    'Value': precip_2050_values.tolist() + temp_2050_values.tolist(),
    'Variable': ['Precipitation'] * len(precip_2050_values) + ['Temperature'] * len(temp_2050_values),
    'Year': ['2050'] * (len(precip_2050_values) + len(temp_2050_values))
})

# Combine both dataframes
df_violin = pd.concat([data_2020, data_2050])

# Plot violin plots
plt.figure(figsize=(12, 6))
sns.violinplot(x='Variable', y='Value', hue='Year', data=df_violin, split=True, palette=['royalblue', 'darkorange'])

# Customize the plot
plt.title('Violin Plots of Precipitation and Temperature (2020 vs. 2050)', fontsize=14)
plt.xlabel('Climate Variables', fontsize=12)
plt.ylabel('Values', fontsize=12)
plt.legend(title="Year")

# Show the plot
plt.tight_layout()
plt.show()


#### Spatial Distribution Maps (2020 vs. 2050)

In [ ]:
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import geopandas as gpd

# Load Nepal boundary shapefile
nepal_shapefile = "data/nepal_shapefiles/local_unit.shp"
gdf = gpd.read_file(nepal_shapefile)

# Function to load raster data
def load_raster(raster_path):
    with rasterio.open(raster_path) as src:
        data = src.read(1)
        nodata_value = src.nodata
        data = np.where(data == nodata_value, np.nan, data)
        extent = src.bounds
    return data, extent

# File paths
precip_2020_path = "data/nepal_climate_rasters/nepal_precipitation_2020.tif"
precip_2050_path = "data/nepal_climate_rasters/nepal_precipitation_2050.tif"
temp_2020_path = "data/nepal_climate_rasters/nepal_temperature_2020.tif"
temp_2050_path = "data/nepal_climate_rasters/nepal_temperature_2050.tif"

# Load raster data
precip_2020, extent_p = load_raster(precip_2020_path)
precip_2050, extent_p = load_raster(precip_2050_path)
temp_2020, extent_t = load_raster(temp_2020_path)
temp_2050, extent_t = load_raster(temp_2050_path)

# Create side-by-side plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Precipitation 2020
im1 = axes[0, 0].imshow(precip_2020, cmap="Blues", interpolation="nearest", extent=[extent_p.left, extent_p.right, extent_p.bottom, extent_p.top])
axes[0, 0].set_title("Precipitation (2020)")
gdf.boundary.plot(ax=axes[0, 0], color="black", linewidth=1)
fig.colorbar(im1, ax=axes[0, 0], label="Precipitation (mm)")

# Precipitation 2050
im2 = axes[0, 1].imshow(precip_2050, cmap="Blues", interpolation="nearest", extent=[extent_p.left, extent_p.right, extent_p.bottom, extent_p.top])
axes[0, 1].set_title("Precipitation (2050)")
gdf.boundary.plot(ax=axes[0, 1], color="black", linewidth=1)
fig.colorbar(im2, ax=axes[0, 1], label="Precipitation (mm)")

# Temperature 2020
im3 = axes[1, 0].imshow(temp_2020, cmap="coolwarm", interpolation="nearest", extent=[extent_t.left, extent_t.right, extent_t.bottom, extent_t.top])
axes[1, 0].set_title("Temperature (2020)")
gdf.boundary.plot(ax=axes[1, 0], color="black", linewidth=1)
fig.colorbar(im3, ax=axes[1, 0], label="Temperature (°C)")

# Temperature 2050
im4 = axes[1, 1].imshow(temp_2050, cmap="coolwarm", interpolation="nearest", extent=[extent_t.left, extent_t.right, extent_t.bottom, extent_t.top])
axes[1, 1].set_title("Temperature (2050)")
gdf.boundary.plot(ax=axes[1, 1], color="black", linewidth=1)
fig.colorbar(im4, ax=axes[1, 1], label="Temperature (°C)")

# Adjust layout
plt.tight_layout()
plt.show()


### Correlation between Temperature and Precipitation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Flatten the data
precip_2020_flat = precip_2020_values.flatten()
temp_2020_flat = temp_2020_values.flatten()
precip_2050_flat = precip_2050_values.flatten()
temp_2050_flat = temp_2050_values.flatten()

# Create a DataFrame
data = pd.DataFrame({
    'Precipitation': np.concatenate([precip_2020_flat, precip_2050_flat]),
    'Temperature': np.concatenate([temp_2020_flat, temp_2050_flat]),
    'Year': ['2020'] * len(precip_2020_flat) + ['2050'] * len(precip_2050_flat)
})

# Set up the plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Precipitation', y='Temperature', hue='Year', data=data, palette=['royalblue', 'darkorange'], alpha=0.5)
sns.regplot(x='Precipitation', y='Temperature', data=data[data['Year'] == '2020'], scatter=False, color='blue', line_kws={'label': '2020 Trend'})
sns.regplot(x='Precipitation', y='Temperature', data=data[data['Year'] == '2050'], scatter=False, color='orange', line_kws={'label': '2050 Trend'})

# Customize the plot
plt.title('Scatter Plot with Regression Lines: Precipitation vs Temperature (2020 vs 2050)', fontsize=14)
plt.xlabel('Precipitation (mm)', fontsize=12)
plt.ylabel('Temperature (°C)', fontsize=12)
plt.legend(title="Year")

# Show the plot
plt.tight_layout()
plt.show()


## Discussion

### 1. Statistical Summary of Climate Variables


The fundamental statistical measures for precipitation and temperature in Nepal for the years 2020 and 2050 reveal significant climatic variations over time. Precipitation in 2020 exhibits a mean value of 50.84 mm, with a median of 50.37 mm, suggesting a relatively symmetrical distribution. However, by 2050, the mean precipitation declines to 48.29 mm, accompanied by a broader range (-6.41 mm to 134.97 mm), indicating an increase in extreme rainfall variations. The standard deviation (SD) of precipitation rises from 11.39 to 16.39, underscoring greater variability and potential instability in precipitation patterns.

Conversely, temperature statistics exhibit a clear warming trend. In 2020, the mean temperature is -7.58°C, with a minimum and maximum range spanning -23.2°C to 8.14°C. By 2050, the mean temperature increases to -5.47°C, reflecting an overall warming of 2.11°C. The maximum temperature also rises significantly to 11.47°C, while the standard deviation remains relatively stable at 7.16°C, suggesting that although warming is widespread, its variability does not change drastically. The high correlation coefficient (0.99) between temperature datasets from 2020 and 2050 indicates a persistent and consistent warming pattern across the region.

### 2. Trends and Patterns in Climate Data


A comparative analysis of precipitation and temperature distributions over time reveals several critical trends. The decline in mean precipitation, coupled with an expansion of its range, suggests a shift towards more extreme rainfall events. While some regions may experience prolonged droughts, others may face intensified precipitation, contributing to higher flood risks and water resource instability. This observation aligns with global climate projections that predict increased frequency and severity of hydrological extremes due to climate change.

Temperature trends reinforce the widely observed phenomenon of global warming. The shift toward higher maximum values in 2050, as indicated by the increase from 8.14°C to 11.47°C, suggests a future scenario where Nepal experiences more frequent and intense heatwaves. Such changes could have profound implications for glacier retreat, biodiversity loss, and agricultural productivity, particularly in Nepal’s high-altitude regions, where even minor temperature increases can accelerate glacial melt and disrupt ecosystems.

### 3. Visual Analysis and Interpretation


The histograms of precipitation and temperature distributions for both years provide further insight into the evolving climate patterns. The precipitation distribution for 2020 is relatively uniform, whereas the 2050 histogram displays greater dispersion, indicating more extreme fluctuations in rainfall. The temperature histograms similarly exhibit a noticeable shift towards higher temperatures, supporting the statistical evidence of a warming trend.

Box plots of precipitation and temperature further illustrate these patterns, highlighting an increase in outliers for precipitation in 2050, which suggests a higher frequency of extreme events such as heavy downpours or dry spells. The temperature box plot also confirms a general upward shift in temperature values, with higher maximum values in 2050.

A scatter plot with regression analysis was employed to examine the relationship between precipitation and temperature. In 2020, the correlation coefficient of -0.44 suggests a moderate inverse relationship, where higher precipitation is generally associated with lower temperatures, likely due to the cooling effect of rainfall. However, by 2050, this correlation weakens to -0.28, indicating a possible decoupling of temperature and precipitation trends. This shift could be attributed to changing atmospheric circulation patterns, altered monsoonal dynamics, or increased evaporation rates due to higher temperatures.

### 4. Key Insights and Implications


The exploratory data analysis highlights a clear trajectory of climate change in Nepal, characterized by increasing temperatures and more erratic precipitation patterns. The observed warming of 2.11°C between 2020 and 2050 aligns with global climate projections and suggests that Nepal will likely face increased heat stress, glacial retreat, and shifts in ecological zones. The heightened variability in precipitation underscores the risk of both extreme droughts and intense rainfall events, which could pose severe challenges for water resource management, agriculture, and disaster preparedness.

Furthermore, the weakening correlation between temperature and precipitation suggests a potential shift in Nepal’s climatic system, where traditional monsoon patterns may become less predictable. This finding necessitates further investigation into long-term climate projections, extreme weather modeling, and adaptive strategies to mitigate the potential socio-economic and environmental impacts of these climatic changes.


## Data Sources


1. Desmondonam. (2025). Nepal Climate Change [Dataset]. GitHub. Retrieved March 24, 2025, from https://github.com/Desmondonam/Nepal_Climate_change

2. Government of Nepal. (2020). New political and administrative boundaries shapefile of Nepal [Dataset]. Open Data Nepal. Retrieved March 24, 2025, from https://opendatanepal.com/dataset/new-political-and-administrative-boundaries-shapefile-of-nepal/resource/07b8dc02-a956-461d-8095-3470503f6ae6